# Prompting y comparación de modelos

La evaluación y comparación de modelos en esta etapa, al igual que en la etapa 1, se basan en el análisis de la métrica f1-score macro. Del mismo modo, se usa la métrica de precisión macro y el recall macro para profundizar en el análisis de la métrica principal sobre el rendimiento de los modelos.

### Importaciones

In [1]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import evaluate as e   
import pandas as pd
import pipeline as p
from sklearn.model_selection import train_test_split

## Evaluación modelo etapa 1

Para la evvaluación del modelo de la etapa 1, se usan los datos dados para la etapa 2 como datos de test. Asimismo, se usan los archivos .py creados en el directorio src para facilitar la implementación de la evaluación.

In [2]:
e.evaluate_model_on_file("model_nb_etapa1.pkl","Datos_etapa 2.xlsx","textos","labels")

{'n_samples': 99,
 'accuracy': 0.9090909090909091,
 'precision_macro': 0.9261904761904761,
 'recall_macro': 0.8959763750461424,
 'f1_macro': 0.9063501891984864,
 'f1_micro': 0.9090909090909091,
 'f1_weighted': 0.9089053590601577}

## Prompting

En primer lugar, se hace el cálculo de cuántos datos se cuentan para cada uno de los ODS en el nuevo dataset de la etapa 2.
De este modo, se observa lo siguiente para los datos de la etapa 2: 21 datos para ODS 1, 35 datos para ODS 3 y 43 datos para ODS 4. En este orden de ideas; al tener en cuenta que en los datos de la etapa 1 se tienen 505 datos para ODS 1, 894 para ODS 3 y 1025 para ODS 4; se tienen en total 1068 datos para el ODS 4 que es el de mayor cantidad de datos. Así, el objetivo del prompting realizado es balancear las clases del dataset, de tal forma que, al menos cada clase tenga más de 1000 datos. Asimismo, se espera que, al tener un dataset resultante balanceado, se consiga un modelo con mejores métricas después del reentrenamiento. Por último, es importante mencionar que los datos retornados por el prompting se almacenaron en el archvio "DatosParciales.xlsx" en la carpeta data del proyecto. Asimismo, el archivo usado para el prompting fue el archivo proporcionado en la práctica de generación de datos.

In [3]:
data0 = pd.read_excel("../data/test/Datos_etapa 2.xlsx")
data0.head()

,textos,labels
0,La definición de privación está basada en el m...,1
1,La mayoría de los programas de protección soci...,1
2,Esto es dos veces y media mayor que la partici...,1
3,El enfoque aquí está en los aspectos de la pob...,1
4,El factor educativo representa una mayor contr...,1


In [4]:
data0.shape

(99, 2)

In [5]:
ods_entradas = data0["labels"].value_counts().sort_index()
print("Cuenta del número de entradas por cada ODS")
print(ods_entradas.to_frame("cuenta"))

Cuenta del número de entradas por cada ODS
        cuenta
labels        
1           21
3           35
4           43


In [6]:
data1 = pd.read_excel("../data/train/Datos_etapa1.xlsx")

In [7]:
data2 = pd.read_excel("../data/DatosParciales.xlsx")

A continuación, se unen los datos brindados para la etapa 2, los datos brindados por el prompting y los datos de la etapa 1.

In [8]:
data = pd.concat([data1,data2],ignore_index=True)
data=data[["textos","labels"]]
data.shape

(3168, 2)

In [9]:
data.head()

,textos,labels
0,"""Aprendizaje"" y ""educación"" se consideran sinó...",4
1,Para los niños más pequeños (bebés y niños peq...,4
2,"Además, la formación de especialistas en medic...",3
3,En los países de la OCDE se tiende a pasar de ...,4
4,Este grupo se centró en las personas que padec...,3


De esta forma, al combinar todos los datos con los que se cuenta, se tiene que quedan 1043 datos para el ODS 1, 1057 datos para el ODS 3 y 1068 datos para el ODS 4, por consiguiente, se alcanza el objetivo del prompting de balancear las clases en el dataset. El dataset final se guarda en el archivo "DatosCompletos.xlsx" en la carpeta data del proyecto.

In [10]:
ods_entradas = data["labels"].value_counts().sort_index()
print("Cuenta del número de entradas por cada ODS")
print(ods_entradas.to_frame("cuenta"))

Cuenta del número de entradas por cada ODS
        cuenta
labels        
1         1043
3         1057
4         1068


In [11]:
data.to_excel("../data/DatosCompletos.xlsx")

## Reentrenamiento y evaluación del modelo etapa 2

En primer lugar, se hace la respectiva división de los datos en train y test. Los datos de train se almacenan en el archivo "DatosAumentadosTrain.xlsx" en el subdirectorio train de la carpeta data. Por su parte, los datos de test se guardan en el archivo "DatosAumentadosTest.xlsx" en el subdirectorio test de la carpeta data.

In [12]:
X = data['textos']
y = data['labels']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

train = pd.concat([X_train,y_train], axis=1)
train.to_excel("../data/train/DatosAumentadosTrain.xlsx",index=False)
train.head()

test = pd.concat([X_test,y_test], axis=1)
test.to_excel("../data/test/DatosAumentadosTest.xlsx",index=False)

Como siguiente paso, se reentrena el modelo con los datos aumentados de train mediante el uso de las funcionalidades disponibles en el archivo pipeline.py. Asimismo, una vez se reentrena el modelo, se persiste el modelo nuevo en un archivo .pkl para poder evaluar el modelo con las funcionalidades el archivo evaluate.py.

In [ ]:
modelo2 = p.entrenar_modelo(X_train,y_train)

In [15]:
modelo2_pipe = modelo2[0]
r = p.guardar_modelo(modelo2_pipe,'../models/model_nb_etapa2')
r

-> Modelo guardado en: ../models/model_nb_etapa2_2025-10-13_14-29-18.pkl


'../models/model_nb_etapa2_2025-10-13_14-29-18.pkl'

Por último, como se mencionó previamente, se usa la función evaluate_model_on_file del archivo evaluate.py para evaluar el rendimiento del modelo reentrenado. A esta funcionalidad se le debe pasar como parámetros el modelo como archivo .pkl y los datos de test almacenados en su respectivo archivo de excel.

In [17]:
e.evaluate_model_on_file("model_nb_etapa2_2025-10-13_14-29-18.pkl","DatosAumentadosTest.xlsx",'textos','labels')

{'n_samples': 792,
 'accuracy': 0.9633838383838383,
 'precision_macro': 0.9638686476921771,
 'recall_macro': 0.9634531417754975,
 'f1_macro': 0.9633925230051704,
 'f1_micro': 0.9633838383838383,
 'f1_weighted': 0.9634273087474405}

## Comparación de modelos

|Modelo|Accuracy|Precision macro|Recall macro|f1 macro|
|------|--------|---------------|------------|--------|
|Etapa1|0.9091|0.9262|0.8958|0.9064|
|Etapa2|0.9634|0.9639|0.9634|0.9634|

Previo al análisis de los resultados para comparar los modelos, resulta pertinente aclarar que en la tabla comparativa presentada el modelo Etapa1 corresponde al mejor modelo hallado en la etapa 1 del proyecto y, por su parte, el modelo Etapa2 corresponde al modelo reentrenado con los datos completos y balanceados que se generaron con la combinación de los datos de etapa 1, los datos de etapa 2 y los datos generados por el prompting.
Con esto en mente, se procede al análisis de los resultados.
- Se cumple lo esperado. El modelo reentrenado con más datos (3168 datos en total) y con datos balanceados tiene un mejor rendimiento que el mejor modelo hallado en la etapa 1. 
- En primer lugar, al tener más datos para su entrenamiento, el modelo Etapa2 tiene más recursos para conocer e identificar los patrones y comportamientos relevantes con mayor detalle para predecir de una mejor manera. En otras palabras, al contar con más datos, el modelo tiene mayor información para aprender sobre las características relevantes que permiten identificar con mayor claridad a qué ODS pertenece una opinión.
- En segundo lugar, el balanceo de las clases permite prevenir que el modelo caiga en sesgos que lo hagan muy bueno para predecir opiniones que pertenecen a las clases con mayores datos, pero que lo hagan no tan bueno para predecir las opiniones de los ODS para los que se tienen menos datos. En otras palabras, con el balanceo de clases se consigue un equilibrio del modelo para que aprenda a predecir el ODS al que pertenecen las opiniones correctamente, sin importar a qué ODS pertenecen realmente porque con el balanceo de los datos el modelo aprende por igual a distinguir los detalles relevantes para clasificar acertadamente una opinión.
- Debido a que es mejor el modelo reentrenado, lo apropiado es usar dicho modelo para el despliegue de la aplicación para el usuario.
- Como métrica general, el modelo Etapa2 tienen una exactitud bastante alta, casi igual a 1, lo que indica que en general, teniendo en cuenta que los datos están balanceados, el modelo clasifica más del 96% de las opiniones en su ODS correcto.
- Con base en la precisión macro del modelo de 0.9639, se evidencia un excelente rendimiento del modelo en cuanto a su precisión, ya que la mayoría (más del 96%) de las clasificaciones que hizo para un ODS fueron correctas. Asimismo, según su recall macro, sin importar el ODS, el modelo tiene un excelente rendimiento para detectar las opiniones que pertenecen a dicho ODS.
- Por último, a partir de la métrica de comparación principal que es el f1-score macro, el modelo reentrenado evidencia una clara mejora en el rendimiento respecto al mejor modelo de la etapa 1. En este orden de ideas, el f1 macro de 0.9634 muestra que el modelo reentrenado con más datos y datos balanceados se equivoca poco en sus predicciones y, además, detecta con facilidad el ODS correcto al que pertenece una opinión, sin importar el ODS al que realmente pertenece la opinión.    